In [1]:
#!pip install ucimlrepo
!pip install pandas-profiling

                                              0.0/262.6 kB ? eta -:--:--
     ------------------------------------- 262.6/262.6 kB 15.8 MB/s eta 0:00:00
                                              0.0/309.8 kB ? eta -:--:--
     -------------------------------------- 309.8/309.8 kB 9.7 MB/s eta 0:00:00
                                              0.0/102.4 kB ? eta -:--:--
     -------------------------------------- 102.4/102.4 kB 6.1 MB/s eta 0:00:00
  Using cached phik-0.12.4-cp311-cp311-win_amd64.whl (667 kB)
  Using cached ImageHash-4.3.1-py2.py3-none-any.whl (296 kB)
  Attempting uninstall: joblib
    Found existing installation: joblib 1.3.2
    Uninstalling joblib-1.3.2:
      Successfully uninstalled joblib-1.3.2


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pycaret 3.3.0 requires joblib>=1.2.0, but you have joblib 1.1.1 which is incompatible.
scikit-learn 1.4.1.post1 requires joblib>=1.2.0, but you have joblib 1.1.1 which is incompatible.

[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import pandas as pd
import math
import numpy as np
import matplotlib as plt
import seaborn as sns
from utils import data_report
from pandas_profiling import ProfileReport

c:\Users\Sandia\AppData\Local\Programs\Python\Python311\Lib\site-packages\numba\core\decorators.py:262: NumbaDeprecationWarning: numba.generated_jit is deprecated. Please see the documentation at: https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-generated-jit for more information and advice on a suitable replacement.
  warnings.warn(msg, NumbaDeprecationWarning)
c:\Users\Sandia\AppData\Local\Programs\Python\Python311\Lib\site-packages\visions\backends\shared\nan_handling.py:50: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @nb.jit


PydanticImportError: `BaseSettings` has been moved to the `pydantic-settings` package. See https://docs.pydantic.dev/2.6/migration/#basesettings-has-moved-to-pydantic-settings for more details.

For further information visit https://errors.pydantic.dev/2.6/u/import-error

## Carga de datos

In [30]:
df = pd.read_csv('data/auto-mpg.csv')

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 398 entries, 0 to 397
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   mpg           398 non-null    float64
 1   cylinders     398 non-null    int64  
 2   displacement  398 non-null    float64
 3   horsepower    398 non-null    object 
 4   weight        398 non-null    int64  
 5   acceleration  398 non-null    float64
 6   model year    398 non-null    int64  
 7   origin        398 non-null    int64  
 8   car name      398 non-null    object 
dtypes: float64(3), int64(4), object(2)
memory usage: 28.1+ KB


In [8]:
df

,mpg,cylinders,displacement,horsepower,weight,acceleration,model year,origin,car name
0,18.0,8,307.0,130,3504,12.0,70,1,chevrolet chevelle malibu
1,15.0,8,350.0,165,3693,11.5,70,1,buick skylark 320
2,18.0,8,318.0,150,3436,11.0,70,1,plymouth satellite
3,16.0,8,304.0,150,3433,12.0,70,1,amc rebel sst
4,17.0,8,302.0,140,3449,10.5,70,1,ford torino
...,...,...,...,...,...,...,...,...,...
393,27.0,4,140.0,86,2790,15.6,82,1,ford mustang gl
394,44.0,4,97.0,52,2130,24.6,82,2,vw pickup
395,32.0,4,135.0,84,2295,11.6,82,1,dodge rampage
396,28.0,4,120.0,79,2625,18.6,82,1,ford ranger


In [9]:
data_report(df=df)   

COL_N,mpg,cylinders,displacement,horsepower,weight,acceleration,model year,origin,car name
DATA_TYPE,float64,int64,float64,object,int64,float64,int64,int64,object
MISSINGS (%),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
UNIQUE_VALUES,129,5,82,94,351,95,13,3,305
CARDIN (%),32.41,1.26,20.6,23.62,88.19,23.87,3.27,0.75,76.63


* COL_N, tipo de variable, tratamiento
* mpg :continua, ver si no todos terminan en cero, en caso contrario convertir a int	
* cylinders: discreta, ver los valores unicos y un conteo
* displacement: continua, ver si no todos terminan en cero, en caso contrario convertir a int
* horsepower: discreta, convertir en int
* weight: continua, nada	
* acceleration: continua, nada	
* model year: discreta, (agrupar por decada)nada	
* origin: discreta, nada	
* car name: nominal, eliminar

In [11]:
df.shape

(398, 9)

In [13]:
df.describe(include='all')

,mpg,cylinders,displacement,horsepower,weight,acceleration,model year,origin,car name
count,398.000000,398.000000,398.000000,398,398.000000,398.000000,398.000000,398.000000,398
unique,NaN,NaN,NaN,94,NaN,NaN,NaN,NaN,305
top,NaN,NaN,NaN,150,NaN,NaN,NaN,NaN,ford pinto
freq,NaN,NaN,NaN,22,NaN,NaN,NaN,NaN,6
mean,23.514573,5.454774,193.425879,NaN,2970.424623,15.568090,76.010050,1.572864,NaN
std,7.815984,1.701004,104.269838,NaN,846.841774,2.757689,3.697627,0.802055,NaN
min,9.000000,3.000000,68.000000,NaN,1613.000000,8.000000,70.000000,1.000000,NaN
25%,17.500000,4.000000,104.250000,NaN,2223.750000,13.825000,73.000000,1.000000,NaN
50%,23.000000,4.000000,148.500000,NaN,2803.500000,15.500000,76.000000,1.000000,NaN
75%,29.000000,8.000000,262.000000,NaN,3608.000000,17.175000,79.000000,2.000000,NaN


In [14]:
df.duplicated().sum()

0

In [15]:
df.isna().sum()

mpg             0
cylinders       0
displacement    0
horsepower      0
weight          0
acceleration    0
model year      0
origin          0
car name        0
dtype: int64

## Limpieza

In [29]:
df['mpg'].astype(int)

0      18
1      15
2      18
3      16
4      17
       ..
393    27
394    44
395    32
396    28
397    31
Name: mpg, Length: 398, dtype: int32

In [31]:
df[~df['displacement'].astype(str).str.endswith('.0')].head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,model year,origin,car name
58,25.0,4,97.5,80,2126,17.0,72,1,dodge colt hardtop


In [32]:
df[~df['mpg'].astype(str).str.endswith('.0')].head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,model year,origin,car name
187,17.5,8,305.0,140,4215,13.0,76,1,chevrolet chevelle malibu classic
189,15.5,8,304.0,120,3962,13.9,76,1,amc matador
190,14.5,8,351.0,152,4215,12.8,76,1,ford gran torino
194,22.5,6,232.0,90,3085,17.6,76,1,amc hornet
196,24.5,4,98.0,60,2164,22.1,76,1,chevrolet woody


In [22]:
df.drop(columns='car name', inplace=True)

In [25]:
#pd.set_option(display.max)
for i in df['horsepower']:
    if i.isnumeric() == False:
        print(i)

?
?
?
?
?
?


In [27]:
df['horsepower'] = pd.to_numeric(df['horsepower'], errors='coerce')

In [33]:
df['marca']=df['car name'].apply(lambda x: x.split()[0])

In [36]:
#df['marca'].unique()

array(['chevrolet', 'buick', 'plymouth', 'amc', 'ford', 'pontiac',
       'dodge', 'toyota', 'datsun', 'volkswagen', 'peugeot', 'audi',
       'saab', 'bmw', 'chevy', 'hi', 'mercury', 'opel', 'fiat',
       'oldsmobile', 'chrysler', 'mazda', 'volvo', 'renault', 'toyouta',
       'maxda', 'honda', 'subaru', 'chevroelt', 'capri', 'vw',
       'mercedes-benz', 'cadillac', 'mercedes', 'vokswagen', 'triumph',
       'nissan'], dtype=object)

In [37]:
df['marca'].replace({
    'chevrolet' : 'chevrolet',
    'toyouta' : 'toyota',
    'vw': 'volkswagen',
    'vokswagen' : 'volkswagen',
    'mercedes' : 'mercedes-benz',
    'maxda' : 'mazda',
    'capri' : 'ford',
    'chevy' : 'chevrolet'
}, inplace=True)

In [38]:
df['marca'].unique()

array(['chevrolet', 'buick', 'plymouth', 'amc', 'ford', 'pontiac',
       'dodge', 'toyota', 'datsun', 'volkswagen', 'peugeot', 'audi',
       'saab', 'bmw', 'hi', 'mercury', 'opel', 'fiat', 'oldsmobile',
       'chrysler', 'mazda', 'volvo', 'renault', 'honda', 'subaru',
       'chevroelt', 'mercedes-benz', 'cadillac', 'triumph', 'nissan'],
      dtype=object)